In [1]:
import requests
from numpy import NaN
import pandas as pd
import numpy as np
import re
import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup
import math
import matplotlib.pyplot as plt

In [2]:
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36','From': 'pleaseletmein@gmail.com'}

#r = requests.get('https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html', headers=headers)

In [3]:
# Russian equipment lost per day since start of war split into category
# https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war (info about categories and datasource)
df_RU_equipment = pd.read_csv("data/russia_losses_equipment.csv")
df_RU_equipment['date'] = pd.to_datetime(df_RU_equipment['date'])
df_RU_equipment = df_RU_equipment.set_index(['date', 'day', 'greatest losses direction'])
# df_RU_equipment = df_RU_equipment.drop('greatest losses direction', axis=1)
df_RU_equipment = df_RU_equipment.diff().fillna(df_RU_equipment).fillna(0).astype(int).reset_index()
df_RU_equipment.tail(10)

,date,day,greatest losses direction,aircraft,helicopter,tank,APC,field artillery,MRL,military auto,fuel tank,drone,naval ship,anti-aircraft warfare,special equipment,mobile SRBM system,vehicles and fuel tanks,cruise missiles
399,2023-03-31,401,NaN,0,0,5,3,4,0,0,0,9,0,0,3,0,3,0
400,2023-04-01,402,NaN,0,0,1,4,8,1,0,0,0,0,1,2,0,7,0
401,2023-04-02,403,NaN,0,0,2,5,4,0,0,0,1,0,0,0,0,9,0
402,2023-04-03,404,NaN,0,0,1,7,7,0,0,0,13,0,1,2,0,16,0
403,2023-04-04,405,NaN,0,0,8,6,3,1,0,0,15,0,0,0,0,9,0
404,2023-04-05,406,NaN,0,0,2,6,10,4,0,0,6,0,0,1,0,11,0
405,2023-04-06,407,NaN,0,1,2,8,7,0,0,0,4,0,1,3,0,1,0
406,2023-04-07,408,NaN,0,0,2,3,8,1,0,0,4,0,0,2,0,13,0
407,2023-04-08,409,NaN,1,0,3,4,5,0,0,0,7,0,1,0,0,12,0
408,2023-04-09,410,NaN,0,0,0,4,13,0,0,0,14,0,0,5,0,3,0


In [4]:
# Russian personnel lost per day
# Prisoners column stopped being updated since 2022-04-28
# https://www.kaggle.com/datasets/piterfm/2022-ukraine-russian-war (info about categories and datasource)
df_RU_personnel = pd.read_csv("data/russia_losses_personnel.csv")
df_RU_personnel['date'] = pd.to_datetime(df_RU_personnel['date'])
df_RU_personnel = df_RU_personnel.set_index(['date', 'day', 'personnel*'])
df_RU_personnel = df_RU_personnel.diff().fillna(df_RU_personnel).fillna(0).astype(int).reset_index()
df_RU_personnel.tail(10)

,date,day,personnel*,personnel,POW
399,2023-03-31,401,about,460,0
400,2023-04-01,402,about,630,0
401,2023-04-02,403,about,560,0
402,2023-04-03,404,about,610,0
403,2023-04-04,405,about,530,0
404,2023-04-05,406,about,550,0
405,2023-04-06,407,about,390,0
406,2023-04-07,408,about,480,0
407,2023-04-08,409,about,570,0
408,2023-04-09,410,about,470,0


In [5]:
# total breakdown of visually confirmed Ukrainian losses (have image of destruction) during war
# https://www.kaggle.com/datasets/5481b37f75c2bc8587e527d6e4d6fd7a81ac46e329e1946c8f85210cbe662eb5  (info about categories and datasource)
df_UA_total = pd.read_csv("data/losses_ukraine.csv")
df_UA_total.tail(10)

,equipment,model,sub_model,manufacturer,losses_total,Unnamed: 5,abandoned,abandoned and destroyed,captured,captured and destroyed,damaged,damaged and abandoned,damaged and captured,damaged beyond economical repair,destroyed,scuttled to prevent capture by Russia,sunk,sunk but raised by Russia
280,"Trucks, Vehicles and Jeeps",Armed pickup truck,with KPV HMG,Ukraine,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
281,"Trucks, Vehicles and Jeeps",Armed pickup truck,with M2 HMG,Ukraine,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
282,"Trucks, Vehicles and Jeeps",Armed pickup truck,with ZU-23 AA gun,Ukraine,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
283,"Trucks, Vehicles and Jeeps",Armed tactical buggy,with ATGM,Ukraine,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
284,"Trucks, Vehicles and Jeeps",Ford Ranger,NaN,the United States,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
285,"Trucks, Vehicles and Jeeps",Peugeot P4,NaN,France,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
286,"Trucks, Vehicles and Jeeps",Unknown tanker,NaN,Ukraine,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
287,"Trucks, Vehicles and Jeeps",Unknown truck,NaN,Ukraine,65,NaN,NaN,NaN,4.0,NaN,2.0,1.0,NaN,NaN,58.0,NaN,NaN,NaN
288,"Trucks, Vehicles and Jeeps",Unknown truck,with ZU-23 AA gun,Ukraine,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
289,"Trucks, Vehicles and Jeeps",Unknown vehicle,NaN,Ukraine,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN


In [7]:
# total breakdown of visually confirmed Russian losses (have image of destruction) during war
# https://www.kaggle.com/datasets/5481b37f75c2bc8587e527d6e4d6fd7a81ac46e329e1946c8f85210cbe662eb5  (info about categories and datasource)
df_RU_total = pd.read_csv("data/losses_russia.csv")
df_RU_total.tail(10)

,equipment,model,sub_model,manufacturer,losses_total,abandoned,abandoned and destroyed,captured,captured and destroyed,captured and stripped,damaged,damaged and abandoned,damaged and captured,damaged beyond economical repair,destroyed,destroyed in a non-combat related incident,sunk
328,"Trucks, Vehicles and Jeeps",LuAZ-969,NaN,the Soviet Union,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
329,"Trucks, Vehicles and Jeeps",UAZ Patriot jeep,NaN,Russia,3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
330,"Trucks, Vehicles and Jeeps",UAZ-31514,NaN,Russia,1,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331,"Trucks, Vehicles and Jeeps",UAZ-23632 pickup truck,NaN,Russia,3,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,"Trucks, Vehicles and Jeeps",UAZ-23632-148-64 armed pickup truck,NaN,Russia,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
333,"Trucks, Vehicles and Jeeps",UAZ-394511 ‘Esaul’,NaN,Russia,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN
334,"Trucks, Vehicles and Jeeps",UAZ-515195 'Esaul',NaN,Russia,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
335,"Trucks, Vehicles and Jeeps",Unknown fuel tanker,NaN,Russia,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
336,"Trucks, Vehicles and Jeeps",(Unknown) truck,NaN,Russia,262,1.0,NaN,10.0,1.0,NaN,7.0,2.0,NaN,NaN,241.0,NaN,NaN
337,"Trucks, Vehicles and Jeeps",(Unknown) vehicle,NaN,Russia,45,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,39.0,NaN,NaN
